In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv('heart.csv')

In [3]:
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


Convert all columns to numeric values

In [4]:
df.loc[df['Sex'] == 'M', 'Sex'] = 0
df.loc[df['Sex'] == 'F', 'Sex'] = 1

In [5]:
df.loc[df['ChestPainType'] == 'TA', 'ChestPainType'] = 3
df.loc[df['ChestPainType'] == 'ATA', 'ChestPainType'] = 2
df.loc[df['ChestPainType'] == 'NAP', 'ChestPainType'] = 1
df.loc[df['ChestPainType'] == 'ASY', 'ChestPainType'] = 0

In [6]:
df.loc[df['RestingECG'] == 'ST', 'RestingECG'] = 2
df.loc[df['RestingECG'] == 'LVH', 'RestingECG'] = 1
df.loc[df['RestingECG'] == 'Normal', 'RestingECG'] = 0

In [7]:
df.loc[df['ExerciseAngina'] == 'Y', 'ExerciseAngina'] = 1
df.loc[df['ExerciseAngina'] == 'N', 'ExerciseAngina'] = 0

In [8]:
df.loc[df['ST_Slope'] == 'Down', 'ST_Slope'] = 2
df.loc[df['ST_Slope'] == 'Flat', 'ST_Slope'] = 1
df.loc[df['ST_Slope'] == 'Up', 'ST_Slope'] = 0

In [9]:
# coerce all data types to numerics
df = df.apply(pd.to_numeric, errors='coerce')

In [10]:
df.dtypes

Age                 int64
Sex                 int64
ChestPainType       int64
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG          int64
MaxHR               int64
ExerciseAngina      int64
Oldpeak           float64
ST_Slope            int64
HeartDisease        int64
dtype: object

Separate and normalize the data

In [11]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
x = df.drop(columns=['HeartDisease'])
y = df['HeartDisease']

In [13]:
# Split training and testing data
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=83)

In [14]:
xtrain = xtrain.reset_index(drop=True)
xtest = xtest.reset_index(drop=True)
ytrain = ytrain.reset_index(drop=True)
ytest = ytest.reset_index(drop=True)

In [15]:
# Feature scaling - convert all features to z-scores
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.fit_transform(xtest)

Train the model

In [33]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras

In [34]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

In [35]:
model = Sequential(
    [
        Dense(units=11, activation='linear', name='input'),
        Dense(units=25, activation='sigmoid', name='h1'),
        Dense(units=1, activation='sigmoid', name='out')
    ]
)

In [36]:
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy', 'precision', 'recall'])

In [37]:
model.fit(x=xtrain, y=ytrain, epochs=250)

Epoch 1/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5256 - loss: 0.6918 - precision: 0.8487 - recall: 0.2000   
Epoch 2/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7551 - loss: 0.6285 - precision: 0.8151 - recall: 0.7096 
Epoch 3/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8130 - loss: 0.5649 - precision: 0.8142 - recall: 0.8443 
Epoch 4/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8262 - loss: 0.5172 - precision: 0.8405 - recall: 0.8539 
Epoch 5/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8170 - loss: 0.4730 - precision: 0.8375 - recall: 0.8449 
Epoch 6/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8457 - loss: 0.4311 - precision: 0.8514 - recall: 0.8770 
Epoch 7/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8657 - loss: 0.4038 - precision: 0.8667 - recall: 0.8774 
Epoch 8/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8383 - loss: 0.3921 - precision: 0.8554 - recall: 0.8473 
Epoch 9/250
23

In [38]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (Dense)                        │ (None, 11)                  │             132 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ h1 (Dense)                           │ (None, 25)                  │             300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out (Dense)                          │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,376 (5.38 KB)

 Trainable params: 458 (1.79 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 918 (3.59 KB)

In [40]:
# Evaluate using the test data
loss, acurracy, precision, recall = model.evaluate(xtest, ytest)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8715 - loss: 0.3518 - precision: 0.8724 - recall: 0.9119 
